## Code from github

### stuff to only run once

In [1]:
# !pip install -U torchdata torch jiwer torchvision
!pwd

/Users/User/Downloads/BME450/Final_Project


In [2]:
""" !pip install numpy
!pip install pandas
!pip install pytorch-lightning
!pip install torch
!pip install torchaudio
!pip install torchmetrics
!pip install torchtext==0.17.0
!pip install torchvision
!pip install evaluate
!pip install wandb """

' !pip install numpy\n!pip install pandas\n!pip install pytorch-lightning\n!pip install torch\n!pip install torchaudio\n!pip install torchmetrics\n!pip install torchtext==0.17.0\n!pip install torchvision\n!pip install evaluate\n!pip install wandb '

In [3]:
#!pip install torchtext==0.17.0

In [4]:
'''
!git init .
!git remote add origin https://github.com/rogAKAnn/image-2-latex.git
!git pull origin main
'''

'\n!git init .\n!git remote add origin https://github.com/rogAKAnn/image-2-latex.git\n!git pull origin main\n'

### Good stuff to run



In [5]:
import pandas as pd
from torch.utils.checkpoint import checkpoint
import numpy as np
import json
import re
import torch
from torch import nn, Tensor
import torchvision
from torchvision import transforms as tvt
from abc import ABC, abstractmethod
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import math
import os
import pytorch_lightning as pl
import random
import pytorch_lightning as pl
from torchtext.data.metrics import bleu_score
from evaluate import load
import matplotlib.pyplot as plt


/Users/User/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/User/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <85A36C65-3F71-3C3B-B529-961AE17DBE73> /Users/User/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <44DEDA27-4DE9-3D4A-8EDE-5AA72081319F> /Users/User/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/User/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytre

In [6]:
torch.__version__

'2.2.0'

In [7]:
data_path = 'im2latex_sorted_by_size'
img_path = 'formula_images_processed'

In [8]:
cuda = torch.cuda.is_available()  
device = torch.device('cuda' if cuda else 'cpu')
device

device(type='cpu')

In [9]:
# Define a class 'Text' that inherits from the abstract base class 'ABC'.
class Text(ABC):
    # Constructor initializes three class attributes commonly used in text processing tasks.
    def __init__(self):
        self.pad_id = 0  # Padding ID, used in tensor processing to maintain uniform sequence lengths.
        self.sos_id = 1  # Start of sequence ID, marks the beginning of text data.
        self.eos_id = 2  # End of sequence ID, marks the termination point of the text data.

    # Abstract method 'tokenize' must be implemented by subclasses to convert strings into a sequence of tokens.
    # The method should return a list of tokens.
    @abstractmethod
    def tokenize(self, formula: str):
        pass

    # Method to convert a tensor of IDs back to a text string, excluding tokens after 'eos_id'.
    def int2text(self, x: Tensor):
        return " ".join([self.id2word[i] for i in x if i > self.eos_id])

    # Method to convert a text string into a tensor of integers based on token IDs.
    def text2int(self, formula: str):
        return torch.LongTensor([self.word2id[i] for i in self.tokenize(formula)])

# Define a class 'Text100k' which inherits from the 'Text' class.
class Text100k(Text):
    # Constructor for initializing the class.
    def __init__(self):
        super().__init__() # Call the constructor of the parent class 'Text'.
        # Open the vocabulary file and initialize 'id2word' list where each line represents a unique word.
        self.id2word = json.load(open("Final_Project/100k_vocab.json", "r"))
        # Create a dictionary 'word2id' mapping each word to its index, providing a fast way to retrieve word IDs.
        self.word2id = dict(zip(self.id2word, range(len(self.id2word))))
        # Define a regular expression pattern to tokenize text. This pattern captures various types of tokens.
        self.TOKENIZE_PATTERN = re.compile(
            "(\\\\[a-zA-Z]+)|" + '((\\\\)*[$-/:-?{-~!"^_`\[\]])|' + "(\w)|" + "(\\\\)"
        )
        # Store the number of classes (unique words in vocabulary) in 'n_class'.
        self.n_class = len(self.id2word)

    # Method to tokenize a string into a list of tokens based on the predefined pattern.
    def tokenize(self, formula: str):
        # Use the compiled pattern to find all matches in the input string.
        tokens = re.finditer(self.TOKENIZE_PATTERN, formula)
        # Convert matches to a list of strings.
        tokens = list(map(lambda x: x.group(0), tokens))
        # Filter out any None or empty strings from the tokens list.
        tokens = [x for x in tokens if x is not None and x != ""]
        return tokens

In [10]:
# Define a class 'LatexDataset' that inherits from PyTorch's 'Dataset' class.
class LatexDataset(Dataset):
    def __init__(self, data_path, img_path, data_type: str, n_sample: int = None, dataset="100k"):
        super().__init__()  # Initialize the parent class

        # Ensure the 'data_type' is one of the specified values to avoid logical errors.
        assert data_type in ["train", "test", "validate"], "Not found data type"
        
        # Construct the path to the dataset file, which is in Excel format.
        excel_path = data_path + f"/im2latex_{data_type}.xlsx"
        
        # Load the dataset from the specified Excel file into a DataFrame.
        # The DataFrame contains two columns: 'formula' and 'image'.
        df = pd.read_excel(excel_path)
        
        # If a sample limit is specified, reduce the DataFrame to the first 'n_sample' rows.
        if n_sample:
            df = df.head(n_sample)
        
        # Update the 'image' column in the DataFrame to include the full path to each image.
        df["image"] = df.image.map(lambda x: img_path + "/" + x)
        
        # Convert the DataFrame into a list of dictionaries, where each dictionary represents a row.
        self.walker = df.to_dict("records")
        
        # Define the image transformation sequence; here converting images to grayscale.
        self.transform = tvt.Compose([tvt.Grayscale(),])

    def __len__(self):
        # Return the number of items in the dataset.
        return len(self.walker)

    def __getitem__(self, idx):
        # Retrieve an item by index, which is a dictionary containing 'formula' and 'image' paths.
        item = self.walker[idx]

        # Extract the formula directly.
        formula = item["formula"]
        # Read the image from the file, ensuring it is in a float format.
        image = torchvision.io.read_image(item["image"])
        image = image.to(dtype=torch.float)
        # Normalize the image pixel values to have a maximum of 1.
        image /= image.max()
        # Apply the predefined transformations to the image.
        image = self.transform(image)
        # Return a tuple of the transformed image and its corresponding formula.
        return image, formula

In [11]:
# Define 'LatexPredictDataset' class that extends PyTorch's 'Dataset' for handling prediction images.
# Used to process single prediction or small set of images, applying normalization to prepare it for input.
class LatexPredictDataset(Dataset):
    def __init__(self, predict_img_path: str):
        super().__init__()  # Initialize the parent class
        # Check if a path to the prediction image is provided.
        if predict_img_path:
            # Assert that the image path exists; if not, raise an error.
            assert os.path.exists(predict_img_path), "Image not found"
            # If the image exists, store the path in a list.
            self.walker = [predict_img_path]
        else:
            # If no path is provided, initialize an empty list.
            self.walker = []
        # Define the image transformation to normalize pixel values.
        self.transform = tvt.Compose([tvt.Normalize((0.5), (0.5)),])

    def __len__(self):
        # Return the number of items in the dataset, which is the length of the 'walker' list.
        return len(self.walker)

    def __getitem__(self, idx):
        # Get the image path from 'walker' using the provided index.
        img_path = self.walker[idx]

        # Load the image from the specified path, and ensure it is in float format for processing.
        image = torchvision.io.read_image(img_path)
        image = image.to(dtype=torch.float)
        # Normalize the image pixel values by dividing by the maximum pixel value.
        image /= image.max()
        # Apply the predefined normalization transformation.
        image = self.transform(image)  # transform image to [-1, 1]

        # Return the processed image.
        return image

In [12]:
# Define a class 'DataModule' that extends 'pl.LightningDataModule' for structured data handling in PyTorch Lightning.
class DataModule(pl.LightningDataModule):
    def __init__(self, train_set, val_set, test_set, predict_set, num_workers: int = 1, batch_size = 20, text = None,):
        super().__init__()
        # Initialize dataset partitions and configuration parameters.
        
        self.train_set = train_set  # Training dataset
        self.val_set = val_set      # Validation dataset
        self.test_set = test_set    # Testing dataset
        self.predict_set = predict_set  # Dataset for prediction
        self.batch_size = batch_size    # Batch size for data loaders
        self.text = text                # Instance of a text handling class for preprocessing
        self.num_workers = num_workers  # Number of worker processes for loading data

    def train_dataloader(self):
        # Return a DataLoader for training with specific configurations.
        
        return DataLoader(
            self.train_set,
            shuffle = True,  # Shuffle data every epoch to avoid overfitting
            batch_size = self.batch_size,
            collate_fn = self.collate_fn,  # Custom function to combine data into batches
            drop_last = True,  # Drop the last incomplete batch if it's not a full size
            num_workers = self.num_workers,
            persistent_workers = True,  # Keep workers alive between batches for performance
        )

    def val_dataloader(self):
        # Return a DataLoader for validation without shuffling.
        
        return DataLoader(
            self.val_set,
            shuffle = False,
            batch_size = self.batch_size,
            collate_fn = self.collate_fn,
            num_workers = self.num_workers,
            persistent_workers = True,
        )

    def test_dataloader(self):
        # Return a DataLoader for testing configured similarly to validation.
        
        return DataLoader(
            self.test_set,
            shuffle = False,
            batch_size = self.batch_size,
            collate_fn = self.collate_fn,
            num_workers = self.num_workers,
            persistent_workers = True,
        )

    def predict_dataloader(self):
        # Return a DataLoader for making predictions.
        
        return DataLoader(self.predict_set, shuffle=False, batch_size=self.batch_size,)

    def collate_fn(self, batch):
        # Custom batch preparation function that processes both text and images.
        
        size = len(batch)  # Get batch size
        # Convert formulas from text to integer sequences.
        formulas = [self.text.text2int(i[1]) for i in batch]
        # Compute the length of each formula.
        formula_len = torch.LongTensor([i.size(-1) + 1 for i in formulas])

        # Pad sequences for consistent batch shape and concatenate special tokens.
        formulas = pad_sequence(formulas, batch_first=True)
        sos = torch.zeros(size, 1) + self.text.word2id["<s>"]  # Start of sequence token
        eos = torch.zeros(size, 1) + self.text.word2id["<e>"]  # End of sequence token
        formulas = torch.cat((sos, formulas, eos), dim=-1).to(dtype=torch.long)

        # Prepare images by padding them to the same size for consistent batch input sizes.
        images = [i[0] for i in batch]  # Extract the first element from each tuple in the batch, which are the images.

        # Initialize variables to track the maximum width and height found in the batch.
        max_width, max_height = 0, 0

        # Loop through each image to determine the maximum dimensions.
        for img in images:
            c, h, w = img.size()  # Extract the channel, height, and width of each image.
            max_width = max(max_width, w)  # Update max_width if the current image width is greater.
            max_height = max(max_height, h)  # Update max_height if the current image height is greater.

        # Define a function to pad each image to these maximum dimensions.
        def padding(img):
            c, h, w = img.size()  # Get the current image size.
            # Calculate padding sizes: no padding for top and left, pad right and bottom to reach max dimensions.
            padder = tvt.Pad((0, 0, max_width - w, max_height - h))  # Create a padding operation.
            return padder(img)  # Apply padding to the image.

        # Apply the padding function to each image and stack them into a single tensor for batch processing.
        images = torch.stack(list(map(padding, images))).to(dtype=torch.float)

        # Return the padded images along with their corresponding formulas and formula lengths.
        return images, formulas, formula_len

### Encoder portion

In [13]:
# Define the ConvWithRowEncoder class, inheriting from nn.Module for creating a custom neural network architecture.
class ConvWithRowEncoder(nn.Module):
    def __init__(self, enc_dim: int):
        super().__init__()  # Initialize the parent class, which sets up the network infrastructure.
        
        # Define a sequential container of convolutional layers to encode image features.
        self.feature_encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),  # First convolution layer with 64 filters.
            nn.MaxPool2d(2, stride=2, padding=0),  # Max pooling to reduce spatial dimensions by half.
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Increase channels to 128 for deeper features.
            nn.MaxPool2d(2, stride=2, padding=0),  # Further reduce spatial dimensions.
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # Increase channels to 256.
            nn.BatchNorm2d(256),  # Normalize the features to improve training stability.
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),  # Additional conv layer with same channel size.
            nn.MaxPool2d(kernel_size=(1,2), stride=(1,2), padding=0),  # Reduce width but not height of the feature map.
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),  # Increase to 512 channels for more complex features.
            nn.BatchNorm2d(512),  # Another batch normalization layer.
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1), padding=0),  # Reduce height but not width.
            nn.Conv2d(512, 512, kernel_size=(3,3), stride=1, padding=1),  # Continue with 512 channels.
            nn.BatchNorm2d(512)  # Final batch normalization.
        )

        # LSTM row encoder that processes each row of the feature map individually.
        self.row_encoder = nn.LSTM(512, enc_dim, batch_first=True, bidirectional=True)
        # Output dimension is doubled due to bidirectionality in LSTM.

        self.enc_dim = enc_dim * 2  # Adjust the encoder dimension to account for bidirectional output.

    def forward(self, x: Tensor):
        """
            x: Tensor with shape (bs, c, w, h) representing batch size, channels, width, and height.
        """
        conv_out = self.feature_encoder(x)  # Process the input through the convolutional layers.
        conv_out = conv_out.permute(0, 2, 3, 1)  # Rearrange dimensions to prepare for row-wise processing (bs, w, h, c).

        bs, w, h, c = conv_out.size()  # Extract dimensions for further use.
        rnn_out = []  # Initialize a list to store output from processing each row.
        for row in range(w):
            row_data = conv_out[:, row, :, :]  # Extract each row across the entire batch.
            row_out, (h, c) = self.row_encoder(row_data)  # Process each row through the LSTM.
            rnn_out.append(row_out)  # Append the LSTM output to the list.

        encoder_out = torch.stack(rnn_out, dim=1)  # Stack all row outputs to form a continuous sequence.
        bs, _, _, d = encoder_out.size()  # Re-extract dimensions to reshape the output.
        encoder_out = encoder_out.view(bs, -1, d)  # Flatten the sequence and channel dimensions.

        return encoder_out  # Return the final encoded output.

### Decoder stuff

In [14]:
# Define the Attention class which inherits from nn.Module for creating custom attention mechanisms.
class Attention(nn.Module):
    """
    This class is central to attention mechanisms, where the model dynamically selects parts of the input data (represented by V) most relevant to the current state of the decoder (represented by h). The attention mechanism enhances the model's ability to focus on different parts of the input sequence at each step of the output sequence generation.
    """

    def __init__(self, enc_dim: int = 512, dec_dim: int = 512, attn_dim: int = 512):
        super().__init__()  # Initialize the parent class, setting up the module infrastructure.
        # Linear transformation to project decoder's hidden state into the attention space.
        self.dec_attn = nn.Linear(dec_dim, attn_dim, bias=False)
        # Linear transformation to project encoder outputs into the same attention space.
        self.enc_attn = nn.Linear(enc_dim, attn_dim, bias=False)
        # Linear transformation to compute a scalar attention score for each encoder output.
        self.full_attn = nn.Linear(attn_dim, 1, bias=False)
        # Softmax layer to normalize the attention scores to probabilities.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, h: Tensor, V: Tensor):
        """
        Compute the context vector as a weighted sum of encoder outputs based on attention scores.
        
        Args:
            h (Tensor): The hidden state of the decoder, shape (b, dec_dim).
            V (Tensor): The encoder outputs, shape (b, w * h, enc_dim).

        Returns:
            Tensor: The context vector, shape (b, enc_dim), summarizing relevant encoder outputs for the current decoder state.
        """
        # Project the decoder hidden state to the attention dimension.
        attn_1 = self.dec_attn(h)  # Shape (b, attn_dim)
        # Project each encoder output to the attention dimension.
        attn_2 = self.enc_attn(V)  # Shape (b, w * h, attn_dim)

        # Calculate the attention scores by adding the projected decoder state to each projected encoder output,
        # applying a non-linearity (tanh), and then computing a scalar score for each.
        attn = self.full_attn(torch.tanh(attn_1.unsqueeze(1) + attn_2)).squeeze(2)  # Shape (b, w * h)

        # Apply softmax to normalize the scores to probabilities.
        alpha = self.softmax(attn)  # Shape (b, w * h)

        # Compute the context vector as the weighted sum of encoder outputs (V),
        # where the weights are the attention probabilities.
        context = (alpha.unsqueeze(2) * V).sum(dim=1)  # Shape (b, enc_dim)

        return context  # Return the context vector which summarizes the attended encoder outputs.



In [15]:
# Define the Decoder class which inherits from nn.Module for creating custom neural network architectures.
class Decoder(nn.Module):
    def __init__(
        self,
        n_class: int,  # Number of output classes (e.g., vocabulary size in language tasks)
        emb_dim: int = 80,  # Dimension of the embedding layer
        enc_dim: int = 512,  # Dimension of the encoder's output
        dec_dim: int = 512,  # Dimension of the decoder's output
        attn_dim: int = 512,  # Dimension of the attention layer
        num_layers: int = 1,  # Number of layers in the LSTM
        dropout: float = 0.1,  # Dropout rate for LSTM
        bidirectional: bool = False,  # Whether the LSTM is bidirectional
        sos_id: int = 1,  # Start of sentence token ID
        eos_id: int = 2,  # End of sentence token ID
    ):
        super().__init__()  # Initialize the parent class.
        self.sos_id = sos_id  # Set the start of sentence ID.
        self.eos_id = eos_id  # Set the end of sentence ID.
        self.embedding = nn.Embedding(n_class, emb_dim)  # Embedding layer to convert input tokens into vectors.
        self.attention = Attention(enc_dim, dec_dim, attn_dim)  # Attention mechanism to focus on specific parts of the input.
        self.concat = nn.Linear(emb_dim + enc_dim, dec_dim)  # Linear layer to combine embedding and attention output.
        self.rnn = nn.LSTM(
            dec_dim,  # Input size equal to decoder dimension.
            dec_dim,  # Output size equal to decoder dimension.
            num_layers,  # Number of RNN layers.
            batch_first=True,  # Input and output tensors are provided as (batch, seq, feature).
            bidirectional=bidirectional,  # If True, becomes a bidirectional LSTM.
            dropout=dropout,  # Dropout rate to use between RNN layers.
        )
        self.out = nn.Linear(dec_dim, n_class)  # Linear layer to map RNN outputs to class scores.
        self.logsoftmax = nn.LogSoftmax(dim=-1)  # LogSoftmax for generating probabilities.

        self.apply(self.init_weights)  # Initialize weights of the network.

    def init_weights(self, layer):
        # Initialize weights for better convergence.
        if isinstance(layer, nn.Embedding):
            nn.init.orthogonal_(layer.weight)  # Orthogonal initialization for embeddings.
        elif isinstance(layer, nn.LSTM):
            for name, param in layer.named_parameters():
                if name.startswith("weight"):
                    nn.init.orthogonal_(param)  # Orthogonal initialization for LSTM weights.

    def forward(self, y, encoder_out=None, hidden_state=None):
        """
        Forward pass of the decoder.

        Args:
            y (Tensor): Target input sequences (batch, target_len).
            encoder_out (Tensor): Output from the encoder (batch, enc_dim, width, height).
            hidden_state (tuple): The hidden and cell states of the LSTM (h, c).

        Returns:
            Tuple[Tensor, tuple]: Log probabilities of the output classes and the new hidden states.
        """
        h, c = hidden_state  # Unpack the hidden and cell states.
        embed = self.embedding(y)  # Embed the input sequences.
        attn_context = self.attention(h, encoder_out)  # Apply attention to the encoder output and previous hidden state.

        # Concatenate the last embedding output and the attention context, then project it to the decoder dimension.
        rnn_input = torch.cat([embed[:, -1], attn_context], dim=1)
        rnn_input = self.concat(rnn_input)

        # Unsqueeze the rnn_input to add a sequence dimension and pass through the LSTM.
        rnn_input = rnn_input.unsqueeze(1)
        hidden_state = h.unsqueeze(0), c.unsqueeze(0)
        out, hidden_state = self.rnn(rnn_input, hidden_state)
        # Apply a linear layer and log softmax to get the log probabilities of each class.
        out = self.logsoftmax(self.out(out))
        h, c = hidden_state
        return out, (h.squeeze(0), c.squeeze(0))  # Return the log probabilities and the new hidden states.

### Putting it all together vers 1

In [16]:
# Define the Image2Latex class which inherits from nn.Module.
class Image2Latex(nn.Module):
    def __init__(
        self,
        n_class: int,  # Number of classes, typically the size of the vocabulary.
        enc_dim: int = 512,  # Dimensionality of the encoder's output vectors.
        enc_type: str = "conv_row_encoder",  # Type of encoder to use.
        emb_dim: int = 80,  # Dimensionality of the embeddings.
        dec_dim: int = 512,  # Dimensionality of the decoder's output vectors.
        attn_dim: int = 512,  # Dimensionality for the attention mechanism.
        num_layers: int = 1,  # Number of layers in the decoder's LSTM.
        dropout: float = 0.1,  # Dropout rate for regularization in the LSTM.
        bidirectional: bool = False,  # Whether the LSTM should be bidirectional.
        decode_type: str = "greedy",  # Decoding strategy: 'greedy' or 'beamsearch'.
        text: Text = None,  # Text processing class instance.
        beam_width: int = 5,  # Beam width for beam search decoding.
        sos_id: int = 1,  # Start-of-sequence token ID.
        eos_id: int = 2,  # End-of-sequence token ID.
    ):
        # Assert to check valid encoder type.
        assert enc_type in [
            "conv_row_encoder",
            "conv_encoder",
            "conv_bn_encoder",
            "resnet_encoder",
            "resnet_row_encoder",
        ], "Not found encoder"
        super().__init__()
        self.n_class = n_class
        
        # Conditional initialization for different types of encoders.
        # self.encoder = based on enc_type, one of several possible encoders is initialized.
        self.encoder = ConvWithRowEncoder(enc_dim=enc_dim)  # Example initialization
        enc_dim = self.encoder.enc_dim
        
        # Initialize the decoder with specified parameters.
        self.decoder = Decoder(
            n_class=n_class,
            emb_dim=emb_dim,
            dec_dim=dec_dim,
            enc_dim=enc_dim,
            attn_dim=attn_dim,
            num_layers=num_layers,
            dropout=dropout,
            bidirectional=bidirectional,
            sos_id=sos_id,
            eos_id=eos_id,
        )
        
        # Linear layers to initialize decoder's hidden and cell states from encoder output.
        self.init_h = nn.Linear(enc_dim, dec_dim)
        self.init_c = nn.Linear(enc_dim, dec_dim)
        
        # Assert to check valid decode type.
        assert decode_type in ["greedy", "beamsearch"]
        self.decode_type = decode_type
        self.text = text
        self.beam_width = beam_width

    def init_decoder_hidden_state(self, V: Tensor):
        """
        Initializes the decoder's hidden and cell states based on encoder output.
        
        Args:
            V (Tensor): Encoder output.
            
        Returns:
            tuple: Initialized hidden and cell states.
        """
        encoder_mean = V.mean(dim=1)  # Compute mean of encoder output to use as basis for hidden state.
        h = torch.tanh(self.init_h(encoder_mean))  # Initialize hidden state.
        c = torch.tanh(self.init_c(encoder_mean))  # Initialize cell state.
        return h, c

    def forward(self, x: Tensor, y: Tensor, y_len: Tensor):
        """
        Forward pass through the model.
        
        Args:
            x (Tensor): Input image batch.
            y (Tensor): Target LaTeX sequences.
            y_len (Tensor): Length of each sequence in the batch.
            
        Returns:
            Tensor: Predicted LaTeX sequences.
        """
        encoder_out = self.encoder(x)  # Get encoder output.
        hidden_state = self.init_decoder_hidden_state(encoder_out)  # Initialize hidden state.
        
        # Decode the sequences.
        predictions = []
        for t in range(y_len.max().item()):  # Iterate through the maximum sequence length in the batch.
            dec_input = y[:, t].unsqueeze(1)  # Prepare decoder input for current timestep.
            out, hidden_state = self.decoder(dec_input, encoder_out, hidden_state)  # Decode.
            predictions.append(out.squeeze(1))  # Collect decoder outputs.
        
        predictions = torch.stack(predictions, dim=1)  # Stack predictions along sequence dimension.
        return predictions

    def decode(self, x: Tensor, max_length: int = 150):
        """
        Decodes an image to LaTeX using specified strategy.
        
        Args:
            x (Tensor): Input image.
            max_length (int): Maximum length of the output sequence.
            
        Returns:
            str: Decoded LaTeX sequence.
        """
        predict = []
        if self.decode_type == "greedy":
            predict = self.decode_greedy(x, max_length)  # Perform greedy decoding.
        elif self.decode_type == "beamsearch":
            predict = self.decode_beam_search(x, max_length)  # Perform beam search decoding.
        return self.text.int2text(predict)  # Convert integer sequence to text.

    def decode_greedy(self, x: Tensor, max_length: int = 150):
        """
        Decodes using a greedy algorithm, choosing the most probable next token at each step.
        
        Args:
            x (Tensor): Input image tensor.
            max_length (int): Maximum sequence length to generate.
        
        Returns:
            List[int]: Sequence of token IDs representing the decoded LaTeX expression.
        """
        encoder_out = self.encoder(x)  # Pass the input image through the encoder.
        bs = encoder_out.size(0)  # Batch size, should generally be 1 for decoding.

        hidden_state = self.init_decoder_hidden_state(encoder_out)  # Initialize the hidden state from encoder output.

        y = torch.LongTensor([self.decoder.sos_id]).view(bs, -1)  # Start with the SOS token.

        predictions = []
        for t in range(max_length):  # Iterate through each step up to max_length.
            out, hidden_state = self.decoder(y, encoder_out, hidden_state)  # Decode the current input.

            k = out.argmax(dim=-1)  # Choose the most likely next token.

            predictions.append(k.item())  # Append the predicted token ID to the result list.

            y = torch.LongTensor([k]).view(bs, -1)  # Prepare the next input token.
        return predictions

    def decode_beam_search(self, x: Tensor, max_length: int = 150):
        """
        Decodes using a beam search algorithm, maintaining multiple hypotheses at each step and expanding them.
        
        Args:
            x (Tensor): Input image tensor.
            max_length (int): Maximum sequence length to generate.
        
        Returns:
            List[int]: Sequence of token IDs representing the best decoded LaTeX expression.
        """
        encoder_out = self.encoder(x)  # Pass the input image through the encoder.
        bs = encoder_out.size(0)  # Batch size, should generally be 1 for decoding.

        hidden_state = self.init_decoder_hidden_state(encoder_out)  # Initialize decoder hidden and cell states.

        list_candidate = [
            ([self.decoder.sos_id], hidden_state, 0)  # Start with the SOS token, initial state, and log probability of 0.
        ]

        for t in range(max_length):  # Iterate through each timestep up to max_length.
            new_candidates = []
            for inp, state, log_prob in list_candidate:
                y = torch.LongTensor([inp[-1]]).view(bs, -1).to(device=x.device)  # Prepare the last token as input.
                out, hidden_state = self.decoder(y, encoder_out, state)  # Decode the input.

                topk = out.topk(self.beam_width, dim=-1)  # Get the top 'beam_width' token predictions.
                new_log_prob = topk.values.view(-1).tolist()  # Extract log probabilities.
                new_idx = topk.indices.view(-1).tolist()  # Extract token indices.

                # Expand each candidate with new possible steps.
                for val, idx in zip(new_log_prob, new_idx):
                    new_inp = inp + [idx]  # Append the new token to the sequence.
                    new_candidates.append((new_inp, hidden_state, log_prob + val))  # Store new candidate.

            # Sort new candidates by their log probabilities and keep the best 'beam_width' candidates.
            new_candidates = sorted(new_candidates, key=lambda x: x[2], reverse=True)
            list_candidate = new_candidates[:self.beam_width]  # Prune to maintain only the top beam_width candidates.

        # Return the sequence of token IDs for the highest scoring candidate after completing the search.
        return list_candidate[0][0]

### Putting it all together vers 2

In [17]:
# Define Image2LatexModel which extends from pl.LightningModule for efficient training with PyTorch Lightning.
class Image2LatexModel(pl.LightningModule):
    def __init__(
        self,
        lr,  # Learning rate for the optimizer.
        total_steps,  # Total training steps for learning rate scheduling.
        n_class: int,  # Number of classes in the output (size of the LaTeX vocabulary).
        enc_dim: int = 512,  # Dimensionality of the encoder outputs.
        enc_type: str = "conv_row_encoder",  # Type of encoder to be used.
        emb_dim: int = 80,  # Dimensionality of the embeddings.
        dec_dim: int = 512,  # Dimensionality of the decoder's LSTM outputs.
        attn_dim: int = 512,  # Dimensionality of the attention mechanism.
        num_layers: int = 1,  # Number of layers in the LSTM.
        dropout: float = 0.1,  # Dropout rate to prevent overfitting.
        bidirectional: bool = False,  # Whether the LSTM should be bidirectional.
        decode_type: str = "greedy",  # Decoding strategy: greedy or beam search. Remember that the decode_greedy method implements a straightforward approach to decoding by always choosing the most probable token at each step. The decode_beam_search method, however, maintains multiple hypotheses (or beams) at once, which can potentially result in better decoding outputs by exploring a broader space of possible sequences.
        text: Text = None,  # Text utility class for tokenization and conversion.
        beam_width: int = 5,  # Beam width used in beam search decoding.
        sos_id: int = 1,  # Start of sequence token ID.
        eos_id: int = 2,  # End of sequence token ID.
        log_step: int = 100,  # Steps between logging.
        log_text: bool = False,  # Boolean to log textual outputs.
    ):
        super().__init__()
        self.model = Image2Latex(
            n_class,
            enc_dim,
            enc_type,
            emb_dim,
            dec_dim,
            attn_dim,
            num_layers,
            dropout,
            bidirectional,
            decode_type,
            text,
            beam_width,
            sos_id,
            eos_id,
        )
        self.criterion = nn.CrossEntropyLoss()  # Loss function for training.
        self.lr = lr  # Store the learning rate.
        self.total_steps = total_steps  # Store the total steps for scheduling.
        self.text = text  # Text processing utility.
        self.max_length = 150  # Maximum length of output sequence.
        self.log_step = log_step  # Interval steps for logging.
        self.log_text = log_text  # Enable textual logging.
        self.exact_match = load("exact_match")  # Load an exact match metric calculation module.
        self.save_hyperparameters()  # Save all hyperparameters for logging and reproducibility.

    def configure_optimizers(self):
        # Set up the optimizer and scheduler for training.
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, betas=(0.9, 0.98))
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=self.hparams.lr, total_steps=self.hparams.total_steps),
            'interval': 'step',  # Update the learning rate after every step.
            'frequency': 1,
            'reduce_on_plateau': False,  # Not applicable for OneCycleLR.
            'monitor': None  # Monitoring not required for this scheduler.
        }
        return [optimizer], [scheduler]

    def forward(self, images, formulas, formula_len):
        # Forward pass to compute outputs from inputs.
        return self.model(images, formulas, formula_len)

    def training_step(self, batch, batch_idx):
        # Defines a single iteration within a training loop.
        if batch_idx % 10 == 0:
            torch.cuda.empty_cache()  # Optionally clear cache every 10 steps to save memory.

        images, formulas, formula_len = batch  # Unpack the batch data.

        formulas_in = formulas[:, :-1]  # Input sequences for training (excluding the last token).
        formulas_out = formulas[:, 1:]  # Target sequences for training (excluding the first token).

        outputs = self.model(images, formulas_in, formula_len)  # Get model predictions.

        bs, t, _ = outputs.size()  # Get size of outputs to reshape for loss calculation.
        _o = outputs.reshape(bs * t, -1)  # Flatten outputs.
        _t = formulas_out.reshape(-1)  # Flatten targets.
        loss = self.criterion(_o, _t)  # Calculate cross entropy loss.

        self.log("train loss", loss, sync_dist=True)  # Log the training loss.

        return loss  # Return the loss for backpropagation.

    def validation_step(self, batch, batch_idx):
        # The validation_step is called with each batch of the validation data. This method evaluates the model's 
        # performance against data it hasn't been trained on, helping ensure the model generalizes well.

        # Unpack the batch data. Each 'batch' usually consists of several elements like images, corresponding formulas, 
        # and the lengths of these formulas.
        images, formulas, formula_len = batch

        # Prepare input and target sequences:
        # - `formulas_in` contains all tokens from each sequence except the last one. This is used as input to the decoder.
        # - `formulas_out` contains all tokens from each sequence starting from the second token. This serves as the ground truth.
        formulas_in = formulas[:, :-1]
        formulas_out = formulas[:, 1:]

        # Pass the input sequences along with their corresponding images through the model to generate predictions.
        outputs = self.model(images, formulas_in, formula_len)

        # Determine the size of the outputs tensor which contains the batch size, sequence length, and number of classes.
        bs, t, _ = outputs.size()

        # Reshape the outputs tensor from three dimensions (batch size, sequence length, number of classes) into two dimensions.
        # This flattening step is necessary for loss calculation as the CrossEntropyLoss function expects 2D inputs:
        # - One dimension for the class scores of each token.
        # - One dimension combining batch and sequence lengths.
        _o = outputs.reshape(bs * t, -1)
        _t = formulas_out.reshape(-1)

        # Compute the loss between the outputs of the model and the ground truths for the entire batch.
        loss = self.criterion(_o, _t)

        # Decode predictions to text:
        # This loop decodes each image in the batch to its LaTeX representation using the model's decoding function.
        # The decoding can be based on different strategies like greedy or beam search depending on the model configuration.
        predicts = [
            self.text.tokenize(self.model.decode(i.unsqueeze(0), self.max_length))
            for i in images
        ]
        # Convert ground truth tensor indices to actual text for comparison and metrics calculation.
        truths = [self.text.tokenize(self.text.int2text(i)) for i in formulas]

        # Calculate BLEU-4 score, a common metric for evaluating translations that compares n-grams of the predictions
        # to the n-grams of the ground truth texts and computes precision scores.
        bleu4 = torch.mean(
            torch.Tensor(
                [bleu_score([pre], [[tru]]) for pre, tru in zip(predicts, truths)]
            )
        )

        # Calculate Exact Match (EM), which is a metric that measures the percentage of predictions that exactly match the ground truths.
        em = torch.mean(
            torch.Tensor(
                [
                    torch.tensor(
                        self.exact_match.compute(
                            predictions=[" ".join(pre)], references=[" ".join(tru)]
                        )["exact_match"]
                    )
                    for pre, tru in zip(predicts, truths)
                ]
            )
        )

        # Log additional information about the validation if required:
        if self.log_text and batch_idx % self.log_step == 0:
            # If logging is enabled and the current batch index is a multiple of the logging step, print predictions and truths.
            for truth, pred in zip(truths, predicts):
                print("=" * 20)
                print(f"Truth: [{' '.join(truth)}] | Predict: [{' '.join(pred)}]")
                print("=" * 20)
            print()

        # Use PyTorch Lightning's `self.log` method to record validation loss, BLEU-4 score, and Exact Match for this batch.
        # These logs are useful for monitoring the model during training and can be viewed in tools like TensorBoard.
        self.log("val_loss", loss, sync_dist=True)
        self.log("val_bleu4", bleu4, sync_dist=True)
        self.log("val_exact_match", em, sync_dist=True)

        # Return metrics and loss for further processing or for callbacks.
        return bleu4, em, loss


    def test_step(self, batch, batch_idx):
        # This method processes each batch of test data, similarly to how the validation_step method processes validation data.
        # It's used to measure the model's final performance after all training and validation steps are completed.

        # Unpack the test batch which includes images (the input data), formulas (the target output LaTeX code), and formula_len (the lengths of each formula).
        images, formulas, formula_len = batch

        # Prepare the input and target sequences for the model:
        # `formulas_in` is used as the input to the model and contains all tokens of the formulas except the last one.
        # `formulas_out` serves as the target output for the model and contains all tokens starting from the second token.
        formulas_in = formulas[:, :-1]
        formulas_out = formulas[:, 1:]

        # Forward pass through the model to generate predictions based on the input sequences and associated images.
        outputs = self.model(images, formulas_in, formula_len)

        # Calculate the size of the outputs tensor, which contains the batch size, sequence length, and number of output classes (vocabulary size).
        bs, t, _ = outputs.size()

        # Reshape the outputs tensor to match the expected format for loss calculation:
        # Flatten the batch and sequence length dimensions into a single dimension since the loss function
        # expects a 2D tensor where each row corresponds to the output from one time step of one sequence.
        _o = outputs.reshape(bs * t, -1)
        # Flatten the targets tensor similarly, aligning each target output with the corresponding model output for loss computation.
        _t = formulas_out.reshape(-1)

        # Compute the CrossEntropyLoss between the predicted outputs and the actual target sequences,
        # which quantifies how well the model's predictions match the expected outputs.
        loss = self.criterion(_o, _t)

        # Decode the predicted outputs to human-readable LaTeX format using the model's decode function,
        # which involves converting the model's output token indices back to strings.
        predicts = [
            self.text.tokenize(self.model.decode(i.unsqueeze(0), self.max_length))
            for i in images
        ]
        # Similarly, decode the actual target formulas for comparison and metric evaluation.
        truths = [self.text.tokenize(self.text.int2text(i)) for i in formulas]

        # Calculate the BLEU-4 score, a common metric for evaluating text predictions that measures the overlap of n-grams
        # between the predicted outputs and the ground truth texts, with a focus on precision.
        bleu4 = torch.mean(
            torch.Tensor(
                [bleu_score([pre], [[tru]]) for pre, tru in zip(predicts, truths)]
            )
        )

        # Calculate the Exact Match (EM) score, which indicates the percentage of predictions that are exactly the same as the ground truths.
        em = torch.mean(
            torch.Tensor(
                [
                    torch.tensor(
                        self.exact_match.compute(
                            predictions=[" ".join(pre)], references=[" ".join(tru)]
                        )["exact_match"]
                    )
                    for pre, tru in zip(predicts, truths)
                ]
            )
        )

        # If logging is enabled and this batch index is a multiple of the log step, print the predictions and truths for visual inspection.
        if True and batch_idx % self.log_step == 0:
            for truth, pred in zip(truths, predicts):
                print("=" * 20)
                print(f"Truth: [{' '.join(truth)}] | Predict: [{' '.join(pred)}]")
                print("=" * 20)
            print()

        # Log the computed loss and metrics to PyTorch Lightning, which supports integration with logging frameworks like TensorBoard.
        self.log("test_loss", loss, sync_dist=True)
        self.log("test_bleu4", bleu4, sync_dist=True)
        self.log("test_exact_match", em, sync_dist=True)

        # Return the computed metrics and loss for further processing by PyTorch Lightning.
        return bleu4, em, loss


    # Do things you want here at predict step
    def predict_step(self, batch, batch_idx):
        image = batch[0]  # Assuming batch returns a list of images

        # Decode the image to LaTeX using the model
        latex = self.model.decode(image, self.max_length)
        print(f"Predicted LaTeX: {latex}")

        # Optionally visualize the image and the prediction
        if self.log_text:  # Assuming log_text is used to control visualization/logging
            plt.imshow(image.squeeze(0).cpu().numpy(), cmap='gray')
            plt.title(f"Predicted LaTeX: {latex}")
            plt.show()

        return latex

### Actual code time, no more classes

In [18]:
random_state=12
torch.manual_seed(random_state)
np.random.seed(random_state)

In [19]:
train_set = LatexDataset(
    data_path=data_path,
    img_path=img_path,
    data_type="train",
    n_sample=None,
    dataset="100k",
)
val_set = LatexDataset(
    data_path=data_path,
    img_path=img_path,
    data_type="validate",
    n_sample=None,
    dataset="100k",
)
test_set = LatexDataset(
    data_path=data_path,
    img_path=img_path,
    data_type="test",
    n_sample=None,
    dataset="100k",
)

# Change predict_set to a single image (one at a time)
predict_set = LatexPredictDataset(predict_img_path=img_path + '/1a2b5f7906.png')

In [20]:
emb_dim = 80
dec_dim = 512
enc_dim = 256
attn_dim = 256

In [21]:
lr = 0.001
max_length = 150
log_idx = 300
max_epochs = 5
batch_size = 16
steps_per_epoch = round(len(train_set) / batch_size)
total_steps = steps_per_epoch * max_epochs
num_workers = 2
dm = DataModule(
    train_set,
    val_set,
    test_set,
    predict_set,
    num_workers,
    batch_size,
    text,
)

NameError: name 'text' is not defined

In [ ]:
num_layers = 1
drop_out = 0.2
decode = "beamsearch"
beam_width=5

In [ ]:
model = Image2LatexModel(
    lr=lr,
    total_steps=total_steps,
    n_class=text.n_class,
    enc_dim=enc_dim,
    enc_type="conv_encoder",
    emb_dim=emb_dim,
    dec_dim=dec_dim,
    attn_dim=attn_dim,
    num_layers=num_layers,
    dropout=drop_out,
    sos_id=text.sos_id,
    eos_id=text.eos_id,
    decode_type="beamsearch",
    text=text,
    beam_width=beam_width,
    log_step=100,
    log_text="store_true",
)

In [ ]:
grad_clip=3
accumulate_batch=64
max_epoch=15

In [ ]:
from pytorch_lightning import Trainer
trainer = Trainer(max_epochs=max_epoch, accelerator="mps")
trainer.fit(model, dm)

trainer.test(model, datamodule=dm)

print(trainer.predict(model, datamodule=dm))

### wandb method of visualizing output, not needed

In [ ]:
#!wandb login 7bd045bdb6b667a12584a8fa5561a6e55e687501

wandb: Appending key for api.wandb.ai to your netrc file: /Users/User/.netrc


In [ ]:
grad_clip=3
accumulate_batch=64
max_epoch=15

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor
import wandb

# Define accumulate_grad_batches
accumulate_batch = 64  # Example value
batch_size = 32  # Example value
accumulate_grad_batches = accumulate_batch // batch_size

# Define a function to log metrics and parameters to Weights & Biases
def log_to_wandb(trainer, model, metrics):
    wandb.init(project="im2latex-v3", name="final_epoch", resume=True, id="a2704e74f502dd6dca2df18bb51b9e8deee5ae63")
    wandb.watch(model)
    for key, value in metrics.items():
        wandb.log({key: value})
    wandb.log({"learning_rate": trainer.lr_schedulers[0]["scheduler"].get_last_lr()})

import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor

class YourLightningModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Define layers and operations
        self.linear = nn.Linear(10, 5)  # Example: Linear layer with input size 10 and output size 5

    def forward(self, x):
        # Define forward pass
        return self.linear(x)

    def training_step(self, batch, batch_idx):
        # Define your training logic here
        x, y = batch  # Assuming batch contains input and target tensors
        y_pred = self.forward(x)  # Forward pass
        loss = self.loss_function(y_pred, y)  # Compute loss
        self.log('train_loss', loss)  # Log the training loss
        return loss

    def configure_optimizers(self):
        # Define your optimizer and scheduler here
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

# Define a Learning Rate Monitor callback
lr_monitor = LearningRateMonitor(logging_interval="step")

# Define Trainer
trainer = pl.Trainer(
    max_epochs=max_epoch,
    callbacks=[lr_monitor],
    accelerator="cpu",
    strategy="ddp_fork",
    log_every_n_steps=1,
    gradient_clip_val=0,
    accumulate_grad_batches=accumulate_grad_batches,
    devices=1,
)

# Create LightningModule instance
model = YourLightningModule()

# Train the model
print("=" * 10 + "[Train]" + "=" * 10)
trainer.fit(model, datamodule=dm)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


==========[Train]==========


/Users/User/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=gloo
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

/Users/User/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:105: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
Missing logger folder: /Users/User/Downloads/BME450/Final Project/lightning_logs
/Users/User/anaconda3/lib/python3

Training: 0it [00:00, ?it/s]

/Users/User/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py:222: UserWarning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/ParallelNative.cpp:228.)
  torch.set_num_threads(1)
/Users/User/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py:222: UserWarning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/ParallelNative.cpp:228.)
  torch.set_num_threads(1)
/Users/User/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
ckpt_path = 'model.ckpt'

In [ ]:
wandb_logger = pl.loggers.WandbLogger(
    project="bme450-project", name="please_work", log_model="all", resume=True
)
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval="step")

accumulate_grad_batches = accumulate_batch // batch_size
trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[lr_monitor],
    max_epochs=max_epoch,
    accelerator="auto",
    strategy="dp",
    log_every_n_steps=1,
    gradient_clip_val=0,
    accumulate_grad_batches=accumulate_grad_batches,
    devices=-1,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# Trainworking
print("=" * 10 + "[Train]" + "=" * 10)
trainer.fit(datamodule=dm, model=model, ckpt_path=ckpt_path)

Restoring states from the checkpoint path at model.ckpt


==========[Train]==========


UnpicklingError: invalid load key, 'v'.

In [ ]:
# test
print("=" * 10 + "[Validate]" + "=" * 10)
trainer.test(datamodule=dm, model=model, ckpt_path = ckpt_path)

In [ ]:
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('hcmut-cse-project/image2latex-v3/model-236davbe:v2', type='model')
# artifact_dir = artifact.download()

In [ ]:
# ls

In [ ]:
# # test
# print("=" * 10 + "[Test]" + "=" * 10)
# trainer.test(datamodule=dm, model=model, ckpt_path=ckpt_path)

In [ ]:
# # Predict
# print("=" * 10 + "[Predict]" + "=" * 10)
# trainer.predict(datamodule=dm, model=model, ckpt_path=ckpt_path)

## My code

### Overview of the problem

The task at hand involves creating a machine learning model capable of converting images of mathematical equations into their corresponding LaTeX representations. The existing codebase employs a transformer architecture with a convolutional neural network (CNN) encoder and a bidirectional long short-term memory (LSTM) decoder. The goal is to enhance this model by replacing the LSTM decoder with a transformer-based decoder, akin to those used in large language models. This change aims to leverage the transformer's ability to handle sequential data more effectively, potentially improving the model's accuracy and efficiency in translating complex mathematical expressions. Additionally, there's a requirement to rewrite the code to ensure originality and avoid plagiarism.

### Key challenges in solving the problem

1. **Architecture Integration**: Seamlessly integrating a transformer-based decoder with the existing CNN encoder poses a significant challenge, given the differences in how these architectures process and represent data.
2. **Data Representation**: Ensuring that the transformer decoder can effectively handle and interpret the feature representations produced by the CNN encoder is crucial. This involves careful design of the interface between the two components.
3. **Sequence Modeling**: Mathematical equations can be highly complex and hierarchical. The transformer decoder must be capable of capturing these intricacies to accurately generate LaTeX code.
4. **Avoiding Plagiarism**: Rewriting the code to maintain originality while preserving or enhancing its functionality requires a deep understanding of the underlying algorithms and a creative approach to problem-solving.

### Solution 1: Direct Replacement with Pre-trained Transformer Decoder

The first solution involves directly replacing the LSTM decoder with a pre-trained transformer decoder, such as GPT or a variant thereof. This approach leverages the powerful capabilities of large language models that have been trained on diverse datasets, including potentially mathematical expressions.

### Solution 2: Custom Transformer Decoder

The second solution entails designing a custom transformer decoder tailored specifically for the task of decoding mathematical expressions from image features. This approach allows for fine-tuning the architecture, attention mechanisms, and training process to the specific nuances of mathematical LaTeX generation.

### Solution 3: Hybrid Transformer

A third solution could be a hybrid approach, where the transformer decoder is initially seeded with weights from a pre-trained model but is significantly modified or extended to better suit the task. This could involve adding specialized layers or attention mechanisms that are more adept at handling the structured nature of mathematical equations.

### Solution 1 Analysis

**Pros**: Utilizing a pre-trained transformer model can significantly reduce development time and leverage the extensive knowledge these models have acquired. It may also improve the model's performance due to the sophisticated understanding of language these models possess.

**Cons**: Pre-trained models are large and may not be optimized for the specific task of decoding mathematical expressions, potentially leading to inefficiencies. There's also a risk that the model's pre-existing biases or knowledge could interfere with accurate LaTeX generation.

### Solution 2 Analysis

**Pros**: A custom transformer decoder can be optimized for the task, potentially leading to better performance and efficiency. It allows for greater control over the model's architecture and training, enabling the incorporation of domain-specific knowledge.

**Cons**: Designing and training a custom transformer from scratch is time-consuming and requires significant expertise. There's also a risk of overfitting to the training data if not carefully managed.

### Solution 3 Analysis

**Pros**: The hybrid approach combines the strengths of pre-trained models with the flexibility of custom architectures, potentially offering a balance between performance and efficiency. It allows for incremental improvements and fine-tuning.

**Cons**: This approach can be complex to implement, as it requires careful consideration of which aspects of the pre-trained model to retain and what customizations to add. It may also inherit some of the drawbacks of both pre-trained and custom models.

### Additional Solution: Modular Transformer Decoder

An additional solution could involve developing a modular transformer decoder architecture that allows for plug-and-play components specifically designed for mathematical expression decoding. This could include modules for spatial attention over the CNN encoder's output, specialized positional encodings for mathematical symbols, and custom training regimes that focus on the hierarchical nature of LaTeX code.

### Recommendation

Given the complexity and specificity of the task, the hybrid transformer approach seems most promising. It offers a practical balance between leveraging the advanced capabilities of pre-trained models and the flexibility to adapt to the unique requirements of decoding mathematical expressions into LaTeX. This strategy allows for starting with a strong foundation and iteratively refining the model to achieve optimal performance. Additionally, it provides a pathway for originality in the implementation, addressing the concern of avoiding plagiarism by incorporating unique modifications and extensions tailored to the task.

### Code time

In [19]:
# Required imports
import torch
from torch import nn, Tensor
import torchvision
from torchvision import transforms as tvt
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import json
import os
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.optim.lr_scheduler import OneCycleLR
from transformers import GPT2Model, GPT2Config
import torch.multiprocessing as mp

In [20]:
# Data handling classes remain largely unchanged but are adapted for clarity and efficiency
class LatexDataset(Dataset):
    def __init__(self, data_path, img_path, data_type: str, n_sample: int = None):
        assert data_type in ["train", "test", "validate"], "Invalid data type specified"
        self.data_frame = pd.read_excel(f"{data_path}/im2latex_{data_type}.xlsx")
        if n_sample:
            self.data_frame = self.data_frame.head(n_sample)
        self.data_frame["image"] = self.data_frame["image"].apply(lambda x: f"{img_path}/{x}")
        self.transform = tvt.Compose([tvt.Grayscale(), tvt.ToTensor(), tvt.Normalize((0.5,), (0.5,))])

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        row = self.data_frame.iloc[idx]
        image = torchvision.io.read_image(row["image"]).float()
        image = self.transform(image)
        formula = row["formula"]
        return image, formula

In [21]:
# Text processing for LaTeX formulas
class TextProcessor:
    def __init__(self, vocab_file):
        self.id2word = json.load(open(vocab_file, "r"))
        self.word2id = {w: i for i, w in enumerate(self.id2word)}
        self.pad_id = 0
        self.sos_id = 1
        self.eos_id = 2

    def encode(self, formula: str):
        tokens = formula.split()
        return [self.sos_id] + [self.word2id.get(token, self.word2id['<unk>']) for token in tokens] + [self.eos_id]

    def decode(self, token_ids: list):
        return ' '.join([self.id2word[i] for i in token_ids if i > self.eos_id])

In [22]:
# Transformer Decoder Module
class TransformerDecoder(nn.Module):
    def __init__(self, n_class: int, d_model: int = 512, nhead: int = 8, num_decoder_layers: int = 6, dropout: float = 0.1):
        super().__init__()
        self.embedding = nn.Embedding(n_class, d_model)
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        self.fc_out = nn.Linear(d_model, n_class)

    def forward(self, tgt, memory):
        tgt = self.embedding(tgt)
        output = self.transformer_decoder(tgt, memory)
        return self.fc_out(output)

In [23]:
# Image2Latex Model with Transformer Decoder
class Image2LatexModel(pl.LightningModule):
    def __init__(self, vocab_size, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=0)  # Assuming 0 is the padding index

    def forward(self, images, formulas):
        encoder_outputs = self.encoder(images)
        decoder_outputs = self.decoder(formulas[:-1], encoder_outputs)
        return decoder_outputs

    def training_step(self, batch, batch_idx):
        images, formulas = batch
        decoder_outputs = self(images, formulas)
        loss = self.loss_fn(decoder_outputs.view(-1, decoder_outputs.size(-1)), formulas.view(-1))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=5e-5)
        scheduler = OneCycleLR(optimizer, max_lr=5e-4, total_steps=1000)
        return [optimizer], [{'scheduler': scheduler, 'interval': 'step'}]

In [24]:
# Assuming an encoder based on a pre-trained model or a custom CNN architecture
class ConvEncoder(nn.Module):
    def __init__(self, output_dim=512):
        super().__init__()
        self.conv_layers = torchvision.models.resnet18(pretrained=True)
        self.conv_layers.fc = nn.Linear(self.conv_layers.fc.in_features, output_dim)

    def forward(self, x):
        return self.conv_layers(x)

In [27]:
# DataModule for handling data loading and batching
class LatexDataModule(pl.LightningDataModule):
    def __init__(self, data_path, img_path, batch_size=32, num_workers=4, text_processor=None):
        super().__init__()
        self.data_path = data_path
        self.img_path = img_path
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.text_processor = text_processor

    def setup(self, stage=None):
        self.train_dataset = LatexDataset(self.data_path, self.img_path, "train")
        self.val_dataset = LatexDataset(self.data_path, self.img_path, "validate")
        self.test_dataset = LatexDataset(self.data_path, self.img_path, "test")

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def collate_fn(self, batch):
        images, formulas = zip(*batch)
        images = torch.stack(images, dim=0)
        
        formula_lengths = [len(formula) for formula in formulas]
        max_length = max(formula_lengths)
        padded_formulas = [self.text_processor.encode(formula) + [self.text_processor.pad_id] * (max_length - len(formula)) for formula in formulas]
        padded_formulas_tensor = torch.tensor(padded_formulas, dtype=torch.long)
        
        return images, padded_formulas_tensor  

In [28]:
# Main training routine
def main():
    mp.set_start_method('spawn', force=True)

    data_path = 'im2latex_sorted_by_size'
    img_path = 'formula_images_processed'
    batch_size = 16
    num_workers = 4
    vocab_file = '100k_vocab.json'

    text_processor = TextProcessor(vocab_file)
    vocab_size = len(text_processor.id2word)

    encoder = ConvEncoder(output_dim=512)
    decoder = TransformerDecoder(n_class=vocab_size)

    model = Image2LatexModel(vocab_size=vocab_size, encoder=encoder, decoder=decoder)
    data_module = LatexDataModule(data_path, img_path, batch_size=batch_size, num_workers=num_workers, text_processor=text_processor)

    trainer = pl.Trainer(accelerator='auto', num_nodes=1, max_epochs=10)
    trainer.fit(model, data_module)

if __name__ == "__main__":
    main()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type               | Params
-----------------------------------------------
0 | encoder | ConvEncoder        | 11.4 M
1 | decoder | TransformerDecoder | 25.8 M
2 | loss_fn | CrossEntropyLoss   | 0     
-----------------------------------------------
37.2 M    Trainable params
0         Non-trainable params
37.2 M    Total params
148.785   Total estimated model params size (MB)
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/User/anaconda3/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/User/anaconda3/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: C

In [ ]:
import torch
import pytorch_lightning as pl
import lightning

print("PyTorch version:", torch.__version__)
print("PyTorch Lightning version:", pl.__version__)
print("Lightning module version:", lightning.__version__)

PyTorch version: 2.2.0
PyTorch Lightning version: 2.2.0
Lightning module version: 2.2.2
